In [1]:
import selenium
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import numpy as np
import re
import time
import json

options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')

# Scrape data

In [2]:
url = 'https://www.aidedd.org/dnd-filters/monsters.php#pied'
driver = webdriver.Chrome(options=options)
driver.get(url)

In [3]:
# all_names = []
# rows = driver.find_elements_by_class_name('nocel')
# for row in rows[1:-2]:
#     html = row.get_attribute('innerHTML')
#     name = str(html).split("'")[1]
#     all_names.append(name)
    
# for i in driver.find_elements_by_tag_name('strong')[1:]:
#     if i.text not in all_names:
#         all_names.append(i.text) 

# all_names = sorted(all_names)
# len(all_names)

In [51]:
kaggle_df = pd.DataFrame(pd.read_csv('../../../5e_monsters/resources/dnd_monsters.csv'))
monster_list = kaggle_df['name'].unique()
len(monster_list)

762

In [5]:
url_base = 'https://www.aidedd.org/dnd/monstres.php?vo='

In [6]:
def browser(name):
    driver = webdriver.Chrome(options=options)
    driver.get(url_base+name)
    return driver

In [7]:
all_cards = {}
def main(mon_list):
    '''add card data to list'''
    for name in mon_list:
        name = name.replace(' ','-').replace('(','')\
                                    .replace(')','')\
                                    .replace('/','-')\
                                    .replace("'",'-')
        driver = browser(name)
           
        # get card html
        card_element = driver.find_element_by_class_name('sansSerif').text

        all_cards[name] = card_element
        
        driver.quit()
        
        print(name)

main(monster_list)

aarakocra
abjurer
aboleth
abominable-yeti
acererak
acolyte
adult-black-dragon
adult-blue-dracolich
adult-blue-dragon
adult-brass-dragon
adult-bronze-dragon
adult-copper-dragon
adult-gold-dragon
adult-green-dragon
adult-kruthik
adult-oblex
adult-red-dragon
adult-silver-dragon
adult-white-dragon
air-elemental
air-elemental-myrmidon
albino-dwarf-warrior
aldani
alhoon
alkilith
allip
allosaurus
ambush-drake
amnizu
ancient-black-dragon
ancient-blue-dragon
ancient-brass-dragon
ancient-bronze-dragon
ancient-copper-dragon
ancient-gold-dragon
ancient-green-dragon
ancient-red-dragon
ancient-shadow
ancient-silver-dragon
ancient-white-dragon
androsphinx
animated-armor
ankheg
ankylosaurus
annis-hag
ape
apprentice-wizard
arcanaloth
archdruid
archer
archmage
armanite
assassin
assassin-vine
astral-dreadnought
atropal
aurochs
autumn-eladrin
awakened-shrub
awakened-tree
axe-beak
azer
babau
baboon
badger
bael
balhannoth
balor
banderhobb
bandit
bandit-captain
banshee
baphomet
barbed-devil
bard
barghest
bar

ultroloth
umber-hulk
unicorn
uthgardt-shaman
vajra-safahr
vampire
vampire-spawn
vampiric-mist
vargouille
vegepygmy
vegepygmy-chief
velociraptor
venerable-shadow
venom-troll
veteran
vine-blight
violet-fungus
vrock
vulture
war-priest
warhorse
warhorse-skeleton
warlock-of-the-archfey
warlock-of-the-fiend
warlock-of-the-great-old-one
warlord
warrior-lvl-1
warrior-lvl-2
warrior-lvl-3
warrior-lvl-4
warrior-lvl-5
warrior-lvl-6
wastrilith
water-elemental
water-elemental-myrmidon
water-weird
weasel
werebear
wereboar
wererat
weretiger
werewolf
white-abishai
white-dragon-wyrmling
wight
wild-dog
wild-dog-alpha
will-o--wisp
winged-kobold
winter-eladrin
winter-wolf
wolf
wood-woad
worg
wraith
wyvern
xorn
xvart
xvart-warlock-of-raxivort
yagnoloth
yan-c-bin
yeenoghu
yellow-musk-creeper
yellow-musk-zombie
yeth-hound
yeti
yochlol
young-black-dragon
young-blue-dragon
young-brass-dragon
young-bronze-dragon
young-copper-dragon
young-gold-dragon
young-green-dragon
young-kruthik
young-red-dragon
young-red-sha

In [8]:
# with open('all_stats.txt', 'w') as outfile:
#     json.dump(all_cards, outfile)

In [9]:
# with open('all_stats.txt') as f:
#     json_data = json.load(f)

In [10]:
all_stats = pd.DataFrame.from_dict(all_cards, orient='index')
all_stats = all_stats.reset_index().rename(columns={0:'card_text','index':'monster_name'})
all_stats

,monster_name,card_text
0,aarakocra,"Medium humanoid (aarakocra), neutral good\nArm..."
1,abjurer,"Medium humanoid (any race), any alignment\nArm..."
2,aboleth,"Large aberration, lawful evil\nArmor Class 17 ..."
3,abominable-yeti,"Huge monstrosity, chaotic evil\nArmor Class 15..."
4,acererak,"Medium undead, neutral evil\nArmor Class 21 (n..."
...,...,...
757,yuan-ti-pureblood,"Medium humanoid (yuan-ti), neutral evil\nArmor..."
758,zaratan,"Gargantuan elemental, neutral\nArmor Class 21 ..."
759,zariel,"Large fiend (devil), lawful evil\nArmor Class ..."
760,zombie,"Medium undead, neutral evil\nArmor Class 8\nHi..."


In [11]:
all_stats.to_csv('../../output/all_stats_aidedd2.csv')

# String manipulation

In [12]:
all_blocks = []
for index, row in all_stats.iterrows():
    monster_block = {'name':row.monster_name}
    attribute_split = row.card_text.split('XP)\n')
    monster_block['stats'] = attribute_split[0]
    monster_block['attributes'] = attribute_split[1].split('ACTIONS')[0]
    legendary_split = row.card_text.split('LEGENDARY ACTIONS\n')
    if 'LEGENDARY ACTIONS' in row.card_text:
        all_leg_actions = legendary_split[1]
        if 'ACTIONS' in legendary_split[0]:
            action_split = legendary_split[0].split('ACTIONS\n')
            monster_block['all_actions'] = action_split[1]
        else:
            monster_block['all_actions'] = ''
    else:
        all_leg_actions = ''
        if 'ACTIONS' in legendary_split[0]:
            action_split = legendary_split[0].split('ACTIONS\n')
            monster_block['all_actions'] = action_split[1]
        else:
            monster_block['all_actions'] = ''
    monster_block['all_leg_actions'] = all_leg_actions
    all_blocks.append(monster_block)
            
mon_blocks_df = pd.DataFrame.from_dict(all_blocks, orient='columns') 
mon_blocks_df

,name,stats,attributes,all_actions,all_leg_actions
0,aarakocra,"Medium humanoid (aarakocra), neutral good\nArm...",Dive Attack. If the aarakocra is flying and di...,"Talon. Melee Weapon Attack: +4 to hit, reach 5...",
1,abjurer,"Medium humanoid (any race), any alignment\nArm...",This full creature's stat block is not available.,,
2,aboleth,"Large aberration, lawful evil\nArmor Class 17 ...",Amphibious. The aboleth can breathe air and wa...,Multiattack. The aboleth makes three tentacle ...,"The aboleth can take 3 legendary actions, choo..."
3,abominable-yeti,"Huge monstrosity, chaotic evil\nArmor Class 15...",This full creature's stat block is not available.,,
4,acererak,"Medium undead, neutral evil\nArmor Class 21 (n...",This full creature's stat block is not available.,,
...,...,...,...,...,...
757,yuan-ti-pureblood,"Medium humanoid (yuan-ti), neutral evil\nArmor...",Innate Spellcasting. The yuan-ti's spellcastin...,Multiattack. The yuan-ti makes two melee attac...,
758,zaratan,"Gargantuan elemental, neutral\nArmor Class 21 ...",This full creature's stat block is not available.,,
759,zariel,"Large fiend (devil), lawful evil\nArmor Class ...",Devil's Sight. Magical darkness doesn't impede...,Multiattack. Zariel attacks twice with her lon...,"Zariel can take 3 legendary actions, choosing ..."
760,zombie,"Medium undead, neutral evil\nArmor Class 8\nHi...",Undead Fortitude. If damage reduces the zombie...,"Slam. Melee Weapon Attack: +3 to hit, reach 5 ...",


In [29]:
# lists of saves and skill
saves = ['str_save','dex_save','con_save','int_save','wis_save','cha_save']
monster_skills = ['history','perception','stealth','persuasion',
              'insight','deception','arcana','religion',
              'acrobatics','athletics','intimidation']

moster_stats = []

for index, row in mon_blocks_df.iterrows():
    monster = {}
    
    # name, size, race, alignment, hp, and ac
    monster['name'] = row['name']
    monster_stats = row.stats.split('\n')
    monster['size'] = monster_stats[0].split(' ')[0].lower()
    if '),' in monster_stats[0]:
        monster['type'] = monster_stats[0].split(' ',1)[1].split('),')[0] + ')'
        monster['alignment'] = monster_stats[0].split('), ')[1]
    else:
        monster['type'] = monster_stats[0].split(' ',1)[1].split(',')[0]
        monster['alignment'] = monster_stats[0].split(', ')[1]
    if len(monster_stats[1].split(' ')) > 2:
        monster['ac'] = monster_stats[1].split(' ')[2]
    else:
        monster['ac'] = ''
    monster['hp'] = monster_stats[2].split(' ')[2]
    monster['cr'] = monster_stats[-1].split(' ')[1]
    
    # break speed into categories 
    speed_types = ['speed','swim','fly','climb','burrow']
    all_speeds = []
    for speed in speed_types:
        row_speeds = monster_stats[3].lower()
        if speed in row_speeds:
            if 'speed swim' in row_speeds:
                speed_dict[speed] = 0
            else:
                speed_num = re.search(rf'{speed} (.*?) ft', row_speeds).group(1)
            monster[speed] = int(speed_num)
        else:
            monster[speed] = 0
            
    # ability scores and mods
    monster['strength'] = monster_stats[5].split(' ')[0]
    monster['str_mod'] = monster_stats[5].split('(')[1][:2]
    monster['dex'] = monster_stats[6].split(' ')[0]
    monster['dex_mod'] = monster_stats[6].split('(')[1][:2]
    monster['con'] = monster_stats[7].split(' ')[0]
    monster['con_mod'] = monster_stats[7].split('(')[1][:2]
    monster['intel'] = monster_stats[8].split(' ')[0]
    monster['int_mod'] = monster_stats[8].split('(')[1][:2]
    monster['wis'] = monster_stats[9].split(' ')[0]
    monster['wis_mod'] = monster_stats[9].split('(')[1][:2]
    monster['cha'] = monster_stats[10].split(' ')[0]
    monster['cha_mod'] = monster_stats[10].split('(')[1][:2]
    
    for save in saves:
        monster[save] = 0
    for skill in monster_skills:
        monster[skill] = 0
    monster['senses'] = ''
    monster['languages'] = ''
    
    # saving throws
    for x in monster_stats[11:-1]:
        if x.startswith('Saving Throws'):
            x = x + ','
            for save in saves:
                if save[:3] in x.lower():
                    numstring = re.search(rf'{save[:3]} +(.*?)\,',\
                                        x.lower()).group(1)
                    monster[save] = int(numstring.replace('+ ',''))
        # skills            
        elif x.startswith('Skills'):
            x = x.replace('.',',').replace(' (',',')
            for skill in monster_skills:
                if skill in x.lower():
                    numstring = x.lower().split(f'{skill} +')[1].split(',')[0]
                    monster[skill] = int(numstring.replace(' ',''))

        # senses     
        elif x.startswith('Senses'):
            monster['senses'] = x.split('Senses ')[1]
            
        # languages     
        elif x.startswith('Languages'):
            monster['languages'] = x.split('Languages ')[1]
            
        
    if row.all_leg_actions == '':
        monster['is_legend'] = 0
    else:
        monster['is_legend'] = 1
        
    monster['attributes'] = row.attributes.replace('\n', ' | ')
    monster['actions'] = row.all_actions.replace('\n', ' | ')
    monster['legendary_actions'] = row.all_leg_actions.replace('\n', ' | ')

    moster_stats.append(monster) 

all_stats_df = pd.DataFrame(moster_stats)

all_stats_df

,name,size,type,alignment,ac,hp,cr,speed,swim,fly,...,religion,acrobatics,athletics,intimidation,senses,languages,is_legend,attributes,actions,legendary_actions
0,aarakocra,medium,humanoid (aarakocra),neutral good,12,13,1/4,20,0,50,...,0,0,0,0,passive Perception 15,"Auran, Aarakocra",0,Dive Attack. If the aarakocra is flying and di...,"Talon. Melee Weapon Attack: +4 to hit, reach 5...",
1,abjurer,medium,humanoid (any race),any alignment,,84,9,30,0,0,...,0,0,0,0,passive Perception 11,any four languages,0,This full creature's stat block is not available.,,
2,aboleth,large,aberration,lawful evil,17,135,10,10,40,0,...,0,0,0,0,"darkvision 120 ft., passive Perception 20","Deep Speech, telepathy 120 ft.",1,Amphibious. The aboleth can breathe air and wa...,Multiattack. The aboleth makes three tentacle ...,"The aboleth can take 3 legendary actions, choo..."
3,abominable-yeti,huge,monstrosity,chaotic evil,15,137,9,40,0,0,...,0,0,0,0,"darkvision 60 ft., passive Perception 15",Yeti,0,This full creature's stat block is not available.,,
4,acererak,medium,undead,neutral evil,21,285,23,30,0,0,...,15,0,0,0,"truesight 120 ft., passive Perception 22","Abyssal, Common, Draconic, Dwarvish, Elvish, G...",0,This full creature's stat block is not available.,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,yuan-ti-pureblood,medium,humanoid (yuan-ti),neutral evil,11,40,1,30,0,0,...,0,0,0,0,"darkvision 60 ft., passive Perception 13","Abyssal, Common, Draconic",0,Innate Spellcasting. The yuan-ti's spellcastin...,Multiattack. The yuan-ti makes two melee attac...,
758,zaratan,gargantuan,elemental,neutral,21,307,22,40,40,0,...,0,0,0,0,"darkvision 60 ft., tremorsense 60 ft., passive...",-,0,This full creature's stat block is not available.,,
759,zariel,large,fiend (devil),lawful evil,21,580,26,50,0,150,...,0,0,0,18,"darkvision 120 ft., passive Perception 26","all, telepathy 120 ft.",1,Devil's Sight. Magical darkness doesn't impede...,Multiattack. Zariel attacks twice with her lon...,"Zariel can take 3 legendary actions, choosing ..."
760,zombie,medium,undead,neutral evil,8,22,1/4,20,0,0,...,0,0,0,0,"darkvision 60 ft., passive Perception 8",understands the languages it knew in life but ...,0,Undead Fortitude. If damage reduces the zombie...,"Slam. Melee Weapon Attack: +3 to hit, reach 5 ...",


In [30]:
# all_stats_df.to_csv('../../output/all_stats_aidedd2.csv', index=False)

In [70]:
df = pd.DataFrame(pd.read_csv('../../output/all_stats_aidedd2.csv'))
df

,name,size,type,alignment,ac,hp,cr,speed,swim,fly,...,religion,acrobatics,athletics,intimidation,senses,languages,is_legend,attributes,actions,legendary_actions
0,aarakocra,medium,humanoid (aarakocra),neutral good,12.0,13,1/4,20,0,50,...,0,0,0,0,passive Perception 15,"Auran, Aarakocra",0,Dive Attack. If the aarakocra is flying and di...,"Talon. Melee Weapon Attack: +4 to hit, reach 5...",NaN
1,abjurer,medium,humanoid (any race),any alignment,NaN,84,9,30,0,0,...,0,0,0,0,passive Perception 11,any four languages,0,This full creature's stat block is not available.,NaN,NaN
2,aboleth,large,aberration,lawful evil,17.0,135,10,10,40,0,...,0,0,0,0,"darkvision 120 ft., passive Perception 20","Deep Speech, telepathy 120 ft.",1,Amphibious. The aboleth can breathe air and wa...,Multiattack. The aboleth makes three tentacle ...,"The aboleth can take 3 legendary actions, choo..."
3,abominable-yeti,huge,monstrosity,chaotic evil,15.0,137,9,40,0,0,...,0,0,0,0,"darkvision 60 ft., passive Perception 15",Yeti,0,This full creature's stat block is not available.,NaN,NaN
4,acererak,medium,undead,neutral evil,21.0,285,23,30,0,0,...,15,0,0,0,"truesight 120 ft., passive Perception 22","Abyssal, Common, Draconic, Dwarvish, Elvish, G...",0,This full creature's stat block is not available.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,yuan-ti-pureblood,medium,humanoid (yuan-ti),neutral evil,11.0,40,1,30,0,0,...,0,0,0,0,"darkvision 60 ft., passive Perception 13","Abyssal, Common, Draconic",0,Innate Spellcasting. The yuan-ti's spellcastin...,Multiattack. The yuan-ti makes two melee attac...,NaN
758,zaratan,gargantuan,elemental,neutral,21.0,307,22,40,40,0,...,0,0,0,0,"darkvision 60 ft., tremorsense 60 ft., passive...",-,0,This full creature's stat block is not available.,NaN,NaN
759,zariel,large,fiend (devil),lawful evil,21.0,580,26,50,0,150,...,0,0,0,18,"darkvision 120 ft., passive Perception 26","all, telepathy 120 ft.",1,Devil's Sight. Magical darkness doesn't impede...,Multiattack. Zariel attacks twice with her lon...,"Zariel can take 3 legendary actions, choosing ..."
760,zombie,medium,undead,neutral evil,8.0,22,1/4,20,0,0,...,0,0,0,0,"darkvision 60 ft., passive Perception 8",understands the languages it knew in life but ...,0,Undead Fortitude. If damage reduces the zombie...,"Slam. Melee Weapon Attack: +3 to hit, reach 5 ...",NaN


In [71]:
all_senses = []
for index, row in df.iterrows():
    mon_senses = {'name':row['name'],'passive_perception':0,'darkvision':0,
                  'truesight':0,'tremorsense':0,'blindsight':0}
    row_senses = row.senses.replace('passive Perception', 'passive_perception').split(', ')
    for sense in row_senses:
        if 'passive_perception' in sense:
            mon_senses['passive_perception'] = int(sense.split(' ')[1])
        elif 'darkvision' in sense:
            mon_senses['darkvision'] = int(sense.split(' ')[1])
        elif 'truesight' in sense:
            mon_senses['truesight'] = int(sense.split(' ')[1])       
        elif 'tremorsense' in sense:
            mon_senses['tremorsense'] = int(sense.split(' ')[1])
        elif 'blindsight' in sense:
            mon_senses['blindsight'] = int(sense.split(' ')[1]) 
            
    all_senses.append(mon_senses)
all_senses

[{'name': 'aarakocra',
  'passive_perception': 15,
  'darkvision': 0,
  'truesight': 0,
  'tremorsense': 0,
  'blindsight': 0},
 {'name': 'abjurer',
  'passive_perception': 11,
  'darkvision': 0,
  'truesight': 0,
  'tremorsense': 0,
  'blindsight': 0},
 {'name': 'aboleth',
  'passive_perception': 20,
  'darkvision': 120,
  'truesight': 0,
  'tremorsense': 0,
  'blindsight': 0},
 {'name': 'abominable-yeti',
  'passive_perception': 15,
  'darkvision': 60,
  'truesight': 0,
  'tremorsense': 0,
  'blindsight': 0},
 {'name': 'acererak',
  'passive_perception': 22,
  'darkvision': 0,
  'truesight': 120,
  'tremorsense': 0,
  'blindsight': 0},
 {'name': 'acolyte',
  'passive_perception': 12,
  'darkvision': 0,
  'truesight': 0,
  'tremorsense': 0,
  'blindsight': 0},
 {'name': 'adult-black-dragon',
  'passive_perception': 21,
  'darkvision': 120,
  'truesight': 0,
  'tremorsense': 0,
  'blindsight': 60},
 {'name': 'adult-blue-dracolich',
  'passive_perception': 24,
  'darkvision': 120,
  'tr

In [72]:
df_senses = pd.DataFrame(all_senses)
df = pd.merge(df, df_senses, how='left', on='name')
df

,name,size,type,alignment,ac,hp,cr,speed,swim,fly,...,languages,is_legend,attributes,actions,legendary_actions,passive_perception,darkvision,truesight,tremorsense,blindsight
0,aarakocra,medium,humanoid (aarakocra),neutral good,12.0,13,1/4,20,0,50,...,"Auran, Aarakocra",0,Dive Attack. If the aarakocra is flying and di...,"Talon. Melee Weapon Attack: +4 to hit, reach 5...",NaN,15,0,0,0,0
1,abjurer,medium,humanoid (any race),any alignment,NaN,84,9,30,0,0,...,any four languages,0,This full creature's stat block is not available.,NaN,NaN,11,0,0,0,0
2,aboleth,large,aberration,lawful evil,17.0,135,10,10,40,0,...,"Deep Speech, telepathy 120 ft.",1,Amphibious. The aboleth can breathe air and wa...,Multiattack. The aboleth makes three tentacle ...,"The aboleth can take 3 legendary actions, choo...",20,120,0,0,0
3,abominable-yeti,huge,monstrosity,chaotic evil,15.0,137,9,40,0,0,...,Yeti,0,This full creature's stat block is not available.,NaN,NaN,15,60,0,0,0
4,acererak,medium,undead,neutral evil,21.0,285,23,30,0,0,...,"Abyssal, Common, Draconic, Dwarvish, Elvish, G...",0,This full creature's stat block is not available.,NaN,NaN,22,0,120,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,yuan-ti-pureblood,medium,humanoid (yuan-ti),neutral evil,11.0,40,1,30,0,0,...,"Abyssal, Common, Draconic",0,Innate Spellcasting. The yuan-ti's spellcastin...,Multiattack. The yuan-ti makes two melee attac...,NaN,13,60,0,0,0
758,zaratan,gargantuan,elemental,neutral,21.0,307,22,40,40,0,...,-,0,This full creature's stat block is not available.,NaN,NaN,15,60,0,60,0
759,zariel,large,fiend (devil),lawful evil,21.0,580,26,50,0,150,...,"all, telepathy 120 ft.",1,Devil's Sight. Magical darkness doesn't impede...,Multiattack. Zariel attacks twice with her lon...,"Zariel can take 3 legendary actions, choosing ...",26,120,0,0,0
760,zombie,medium,undead,neutral evil,8.0,22,1/4,20,0,0,...,understands the languages it knew in life but ...,0,Undead Fortitude. If damage reduces the zombie...,"Slam. Melee Weapon Attack: +3 to hit, reach 5 ...",NaN,8,60,0,0,0


In [73]:
# df.to_csv('../../output/aidedd_senses.csv', index=False)

In [76]:
df = df[['name', 'size', 'type', 'alignment', 'languages', 
         'ac', 'hp', 'cr', 'speed', 'swim', 'fly', 'climb', 
         'burrow', 'passive_perception', 'darkvision', 'truesight', 
         'tremorsense', 'blindsight','strength', 'str_mod', 
         'dex', 'dex_mod', 'con', 'con_mod', 'intel', 'int_mod', 
         'wis', 'wis_mod', 'cha', 'cha_mod', 'str_save', 'dex_save', 
         'con_save', 'int_save', 'wis_save','cha_save', 'history', 
         'perception', 'stealth', 'persuasion', 'insight', 
         'deception', 'arcana', 'religion', 'acrobatics', 
         'athletics', 'intimidation', 'senses', 'attributes',
         'actions', 'legendary_actions']]
df['languages'] = np.where(df['languages']=='-','none',df['languages'])
df

,name,size,type,alignment,languages,ac,hp,cr,speed,swim,...,deception,arcana,religion,acrobatics,athletics,intimidation,senses,attributes,actions,legendary_actions
0,aarakocra,medium,humanoid (aarakocra),neutral good,"Auran, Aarakocra",12.0,13,1/4,20,0,...,0,0,0,0,0,0,passive Perception 15,Dive Attack. If the aarakocra is flying and di...,"Talon. Melee Weapon Attack: +4 to hit, reach 5...",NaN
1,abjurer,medium,humanoid (any race),any alignment,any four languages,NaN,84,9,30,0,...,0,8,0,0,0,0,passive Perception 11,This full creature's stat block is not available.,NaN,NaN
2,aboleth,large,aberration,lawful evil,"Deep Speech, telepathy 120 ft.",17.0,135,10,10,40,...,0,0,0,0,0,0,"darkvision 120 ft., passive Perception 20",Amphibious. The aboleth can breathe air and wa...,Multiattack. The aboleth makes three tentacle ...,"The aboleth can take 3 legendary actions, choo..."
3,abominable-yeti,huge,monstrosity,chaotic evil,Yeti,15.0,137,9,40,0,...,0,0,0,0,0,0,"darkvision 60 ft., passive Perception 15",This full creature's stat block is not available.,NaN,NaN
4,acererak,medium,undead,neutral evil,"Abyssal, Common, Draconic, Dwarvish, Elvish, G...",21.0,285,23,30,0,...,0,22,15,0,0,0,"truesight 120 ft., passive Perception 22",This full creature's stat block is not available.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,yuan-ti-pureblood,medium,humanoid (yuan-ti),neutral evil,"Abyssal, Common, Draconic",11.0,40,1,30,0,...,6,0,0,0,0,0,"darkvision 60 ft., passive Perception 13",Innate Spellcasting. The yuan-ti's spellcastin...,Multiattack. The yuan-ti makes two melee attac...,NaN
758,zaratan,gargantuan,elemental,neutral,none,21.0,307,22,40,40,...,0,0,0,0,0,0,"darkvision 60 ft., tremorsense 60 ft., passive...",This full creature's stat block is not available.,NaN,NaN
759,zariel,large,fiend (devil),lawful evil,"all, telepathy 120 ft.",21.0,580,26,50,0,...,0,0,0,0,0,18,"darkvision 120 ft., passive Perception 26",Devil's Sight. Magical darkness doesn't impede...,Multiattack. Zariel attacks twice with her lon...,"Zariel can take 3 legendary actions, choosing ..."
760,zombie,medium,undead,neutral evil,understands the languages it knew in life but ...,8.0,22,1/4,20,0,...,0,0,0,0,0,0,"darkvision 60 ft., passive Perception 8",Undead Fortitude. If damage reduces the zombie...,"Slam. Melee Weapon Attack: +3 to hit, reach 5 ...",NaN


In [77]:
df.loc[df['cr'] == '1/8','cr'] = 0.125
df.loc[df['cr'] == '1/4','cr'] = 0.25
df.loc[df['cr'] == '12','cr'] = 0.5
df

,name,size,type,alignment,languages,ac,hp,cr,speed,swim,...,deception,arcana,religion,acrobatics,athletics,intimidation,senses,attributes,actions,legendary_actions
0,aarakocra,medium,humanoid (aarakocra),neutral good,"Auran, Aarakocra",12.0,13,0.25,20,0,...,0,0,0,0,0,0,passive Perception 15,Dive Attack. If the aarakocra is flying and di...,"Talon. Melee Weapon Attack: +4 to hit, reach 5...",NaN
1,abjurer,medium,humanoid (any race),any alignment,any four languages,NaN,84,9,30,0,...,0,8,0,0,0,0,passive Perception 11,This full creature's stat block is not available.,NaN,NaN
2,aboleth,large,aberration,lawful evil,"Deep Speech, telepathy 120 ft.",17.0,135,10,10,40,...,0,0,0,0,0,0,"darkvision 120 ft., passive Perception 20",Amphibious. The aboleth can breathe air and wa...,Multiattack. The aboleth makes three tentacle ...,"The aboleth can take 3 legendary actions, choo..."
3,abominable-yeti,huge,monstrosity,chaotic evil,Yeti,15.0,137,9,40,0,...,0,0,0,0,0,0,"darkvision 60 ft., passive Perception 15",This full creature's stat block is not available.,NaN,NaN
4,acererak,medium,undead,neutral evil,"Abyssal, Common, Draconic, Dwarvish, Elvish, G...",21.0,285,23,30,0,...,0,22,15,0,0,0,"truesight 120 ft., passive Perception 22",This full creature's stat block is not available.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,yuan-ti-pureblood,medium,humanoid (yuan-ti),neutral evil,"Abyssal, Common, Draconic",11.0,40,1,30,0,...,6,0,0,0,0,0,"darkvision 60 ft., passive Perception 13",Innate Spellcasting. The yuan-ti's spellcastin...,Multiattack. The yuan-ti makes two melee attac...,NaN
758,zaratan,gargantuan,elemental,neutral,none,21.0,307,22,40,40,...,0,0,0,0,0,0,"darkvision 60 ft., tremorsense 60 ft., passive...",This full creature's stat block is not available.,NaN,NaN
759,zariel,large,fiend (devil),lawful evil,"all, telepathy 120 ft.",21.0,580,26,50,0,...,0,0,0,0,0,18,"darkvision 120 ft., passive Perception 26",Devil's Sight. Magical darkness doesn't impede...,Multiattack. Zariel attacks twice with her lon...,"Zariel can take 3 legendary actions, choosing ..."
760,zombie,medium,undead,neutral evil,understands the languages it knew in life but ...,8.0,22,0.25,20,0,...,0,0,0,0,0,0,"darkvision 60 ft., passive Perception 8",Undead Fortitude. If damage reduces the zombie...,"Slam. Melee Weapon Attack: +3 to hit, reach 5 ...",NaN


In [78]:
df.loc[df['attributes'] == "This full creature's stat block is not available.",
       ['attributes','actions','legendary_actions']] = 'unavailable'
df

,name,size,type,alignment,languages,ac,hp,cr,speed,swim,...,deception,arcana,religion,acrobatics,athletics,intimidation,senses,attributes,actions,legendary_actions
0,aarakocra,medium,humanoid (aarakocra),neutral good,"Auran, Aarakocra",12.0,13,0.25,20,0,...,0,0,0,0,0,0,passive Perception 15,Dive Attack. If the aarakocra is flying and di...,"Talon. Melee Weapon Attack: +4 to hit, reach 5...",NaN
1,abjurer,medium,humanoid (any race),any alignment,any four languages,NaN,84,9,30,0,...,0,8,0,0,0,0,passive Perception 11,unavailable,unavailable,unavailable
2,aboleth,large,aberration,lawful evil,"Deep Speech, telepathy 120 ft.",17.0,135,10,10,40,...,0,0,0,0,0,0,"darkvision 120 ft., passive Perception 20",Amphibious. The aboleth can breathe air and wa...,Multiattack. The aboleth makes three tentacle ...,"The aboleth can take 3 legendary actions, choo..."
3,abominable-yeti,huge,monstrosity,chaotic evil,Yeti,15.0,137,9,40,0,...,0,0,0,0,0,0,"darkvision 60 ft., passive Perception 15",unavailable,unavailable,unavailable
4,acererak,medium,undead,neutral evil,"Abyssal, Common, Draconic, Dwarvish, Elvish, G...",21.0,285,23,30,0,...,0,22,15,0,0,0,"truesight 120 ft., passive Perception 22",unavailable,unavailable,unavailable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,yuan-ti-pureblood,medium,humanoid (yuan-ti),neutral evil,"Abyssal, Common, Draconic",11.0,40,1,30,0,...,6,0,0,0,0,0,"darkvision 60 ft., passive Perception 13",Innate Spellcasting. The yuan-ti's spellcastin...,Multiattack. The yuan-ti makes two melee attac...,NaN
758,zaratan,gargantuan,elemental,neutral,none,21.0,307,22,40,40,...,0,0,0,0,0,0,"darkvision 60 ft., tremorsense 60 ft., passive...",unavailable,unavailable,unavailable
759,zariel,large,fiend (devil),lawful evil,"all, telepathy 120 ft.",21.0,580,26,50,0,...,0,0,0,0,0,18,"darkvision 120 ft., passive Perception 26",Devil's Sight. Magical darkness doesn't impede...,Multiattack. Zariel attacks twice with her lon...,"Zariel can take 3 legendary actions, choosing ..."
760,zombie,medium,undead,neutral evil,understands the languages it knew in life but ...,8.0,22,0.25,20,0,...,0,0,0,0,0,0,"darkvision 60 ft., passive Perception 8",Undead Fortitude. If damage reduces the zombie...,"Slam. Melee Weapon Attack: +3 to hit, reach 5 ...",NaN


In [79]:
df = df.apply(pd.to_numeric, errors='ignore')

In [81]:
df['ac'] = df['ac'].fillna(12.0)
df

,name,size,type,alignment,languages,ac,hp,cr,speed,swim,...,deception,arcana,religion,acrobatics,athletics,intimidation,senses,attributes,actions,legendary_actions
0,aarakocra,medium,humanoid (aarakocra),neutral good,"Auran, Aarakocra",12.0,13,0.25,20,0,...,0,0,0,0,0,0,passive Perception 15,Dive Attack. If the aarakocra is flying and di...,"Talon. Melee Weapon Attack: +4 to hit, reach 5...",NaN
1,abjurer,medium,humanoid (any race),any alignment,any four languages,12.0,84,9,30,0,...,0,8,0,0,0,0,passive Perception 11,unavailable,unavailable,unavailable
2,aboleth,large,aberration,lawful evil,"Deep Speech, telepathy 120 ft.",17.0,135,10,10,40,...,0,0,0,0,0,0,"darkvision 120 ft., passive Perception 20",Amphibious. The aboleth can breathe air and wa...,Multiattack. The aboleth makes three tentacle ...,"The aboleth can take 3 legendary actions, choo..."
3,abominable-yeti,huge,monstrosity,chaotic evil,Yeti,15.0,137,9,40,0,...,0,0,0,0,0,0,"darkvision 60 ft., passive Perception 15",unavailable,unavailable,unavailable
4,acererak,medium,undead,neutral evil,"Abyssal, Common, Draconic, Dwarvish, Elvish, G...",21.0,285,23,30,0,...,0,22,15,0,0,0,"truesight 120 ft., passive Perception 22",unavailable,unavailable,unavailable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,yuan-ti-pureblood,medium,humanoid (yuan-ti),neutral evil,"Abyssal, Common, Draconic",11.0,40,1,30,0,...,6,0,0,0,0,0,"darkvision 60 ft., passive Perception 13",Innate Spellcasting. The yuan-ti's spellcastin...,Multiattack. The yuan-ti makes two melee attac...,NaN
758,zaratan,gargantuan,elemental,neutral,none,21.0,307,22,40,40,...,0,0,0,0,0,0,"darkvision 60 ft., tremorsense 60 ft., passive...",unavailable,unavailable,unavailable
759,zariel,large,fiend (devil),lawful evil,"all, telepathy 120 ft.",21.0,580,26,50,0,...,0,0,0,0,0,18,"darkvision 120 ft., passive Perception 26",Devil's Sight. Magical darkness doesn't impede...,Multiattack. Zariel attacks twice with her lon...,"Zariel can take 3 legendary actions, choosing ..."
760,zombie,medium,undead,neutral evil,understands the languages it knew in life but ...,8.0,22,0.25,20,0,...,0,0,0,0,0,0,"darkvision 60 ft., passive Perception 8",Undead Fortitude. If damage reduces the zombie...,"Slam. Melee Weapon Attack: +3 to hit, reach 5 ...",NaN


In [84]:
df = pd.merge(df, kaggle_df[['name','legendary','source']], how='left', on='name')
df['legendary'] = np.where(df['legendary']=='Legendary',1,0)
df['legendary_actions'] = df['legendary_actions'].fillna('none')
df

,name,size,type,alignment,languages,ac,hp,cr,speed,swim,...,religion,acrobatics,athletics,intimidation,senses,attributes,actions,legendary_actions,legendary,source
0,aarakocra,medium,humanoid (aarakocra),neutral good,"Auran, Aarakocra",12.0,13,0.25,20,0,...,0,0,0,0,passive Perception 15,Dive Attack. If the aarakocra is flying and di...,"Talon. Melee Weapon Attack: +4 to hit, reach 5...",none,0,Monster Manual (BR)
1,abjurer,medium,humanoid (any race),any alignment,any four languages,12.0,84,9,30,0,...,0,0,0,0,passive Perception 11,unavailable,unavailable,unavailable,0,Volo's Guide to Monsters
2,aboleth,large,aberration,lawful evil,"Deep Speech, telepathy 120 ft.",17.0,135,10,10,40,...,0,0,0,0,"darkvision 120 ft., passive Perception 20",Amphibious. The aboleth can breathe air and wa...,Multiattack. The aboleth makes three tentacle ...,"The aboleth can take 3 legendary actions, choo...",1,Monster Manual (SRD)
3,abominable-yeti,huge,monstrosity,chaotic evil,Yeti,15.0,137,9,40,0,...,0,0,0,0,"darkvision 60 ft., passive Perception 15",unavailable,unavailable,unavailable,0,Monster Manual
4,acererak,medium,undead,neutral evil,"Abyssal, Common, Draconic, Dwarvish, Elvish, G...",21.0,285,23,30,0,...,15,0,0,0,"truesight 120 ft., passive Perception 22",unavailable,unavailable,unavailable,0,Adventures (Tomb of Annihilation)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,yuan-ti-pureblood,medium,humanoid (yuan-ti),neutral evil,"Abyssal, Common, Draconic",11.0,40,1,30,0,...,0,0,0,0,"darkvision 60 ft., passive Perception 13",Innate Spellcasting. The yuan-ti's spellcastin...,Multiattack. The yuan-ti makes two melee attac...,none,0,Monster Manual (BR)
758,zaratan,gargantuan,elemental,neutral,none,21.0,307,22,40,40,...,0,0,0,0,"darkvision 60 ft., tremorsense 60 ft., passive...",unavailable,unavailable,unavailable,0,Mordenkainen's Tome of Foes
759,zariel,large,fiend (devil),lawful evil,"all, telepathy 120 ft.",21.0,580,26,50,0,...,0,0,0,18,"darkvision 120 ft., passive Perception 26",Devil's Sight. Magical darkness doesn't impede...,Multiattack. Zariel attacks twice with her lon...,"Zariel can take 3 legendary actions, choosing ...",1,Mordenkainen's Tome of Foes
760,zombie,medium,undead,neutral evil,understands the languages it knew in life but ...,8.0,22,0.25,20,0,...,0,0,0,0,"darkvision 60 ft., passive Perception 8",Undead Fortitude. If damage reduces the zombie...,"Slam. Melee Weapon Attack: +3 to hit, reach 5 ...",none,0,Monster Manual (SRD)


In [85]:
df.to_csv('../../output/aidedd_blocks.csv', index=False)